In [1]:
%matplotlib inline

import os
import sys

from functools import partial

import numpy as np
import matplotlib.pyplot as plt
from hmmlearn import hmm

import cardio.dataset as ds
from cardio import EcgBatch
from cardio.dataset import B, V, F
from cardio.models.hmm import HMModel, prepare_hmm_input

import warnings
warnings.filterwarnings('ignore')

/home/shakka/anaconda3/envs/cs224u/lib/python3.6/site-packages/dicom/__init__.py:53: UserWarning: 
This code is using an older version of pydicom, which is no longer 
maintained as of Jan 2017.  You can access the new pydicom features and API 
by installing `pydicom` from PyPI.
See 'Transitioning to pydicom 1.x' section at pydicom.readthedocs.org 
for more information.

  warnings.warn(msg)
/home/shakka/anaconda3/envs/cs224u/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
def get_annsamples(batch):
    return [ann["annsamp"] for ann in batch.annotation]

def get_anntypes(batch):
    return [ann["anntype"] for ann in batch.annotation]

def expand_annotation(annsamp, anntype, length):
    """Unravel annotation
    """
    begin = -1
    end = -1
    s = 'none'
    states = {'N':0, 'st':1, 't':2, 'iso':3, 'p':4, 'pq':5}
    annot_expand = -1 * np.ones(length)

    for j, samp in enumerate(annsamp):
        if anntype[j] == '(':
            begin = samp
            if (end > 0) & (s != 'none'):
                if s == 'N':
                    annot_expand[end:begin] = states['st']
                elif s == 't':
                    annot_expand[end:begin] = states['iso']
                elif s == 'p':
                    annot_expand[end:begin] = states['pq']
        elif anntype[j] == ')':
            end = samp
            if (begin > 0) & (s != 'none'):
                annot_expand[begin:end] = states[s]
        else:
            s = anntype[j]

    return annot_expand

def prepare_means_covars(hmm_features, clustering, states=[3, 5, 11, 14, 17, 19], num_states=19, num_features=3):
    """This function is specific to the task and the model configuration, thus contains hardcode.
    """
    means = np.zeros((num_states, num_features))
    covariances = np.zeros((num_states, num_features, num_features))
    
    # Prepearing means and variances
    last_state = 0
    unique_clusters = len(np.unique(clustering)) - 1 # Excuding value -1, which represents undefined state
    for state, cluster in zip(states, np.arange(unique_clusters)):
        value = hmm_features[clustering == cluster, :]
        means[last_state:state, :] = np.mean(value, axis=0)
        covariances[last_state:state, :, :] = value.T.dot(value) / np.sum(clustering == cluster)
        last_state = state
        
    return means, covariances

def prepare_transmat_startprob():
    """ This function is specific to the task and the model configuration, thus contains hardcode.
    """
    # Transition matrix - each row should add up tp 1
    transition_matrix = np.diag(19 * [14/15.0]) + np.diagflat(18 * [1/15.0], 1) + np.diagflat([1/15.0], -18)
    
    # We suppose that absence of P-peaks is possible
    transition_matrix[13,14]=0.9*1/15.0
    transition_matrix[13,17]=0.1*1/15.0

    # Initial distribution - should add up to 1
    start_probabilities = np.array(19 * [1/np.float(19)])
    
    return transition_matrix, start_probabilities

In [3]:
SIGNALS_PATH = "hmm_data"
SIGNALS_MASK = os.path.join(SIGNALS_PATH, "*.hea")

In [4]:
index = ds.FilesIndex(path=SIGNALS_MASK, no_ext=True, sort=True)
dtst = ds.Dataset(index, batch_class=EcgBatch)
dtst.cv_split(0.9)

In [5]:
# Identify optimal starting params
template_ppl_inits = (
    ds.Pipeline()
      .init_variable("annsamps", init_on_each_run=list)
      .init_variable("anntypes", init_on_each_run=list)
      .init_variable("hmm_features", init_on_each_run=list)
      .load(fmt='wfdb', components=["signal", "annotation", "meta"], ann_ext='pu1')
      .cwt(src="signal", dst="hmm_features", scales=[4,8,16], wavelet="mexh")
      .standardize(axis=-1, src="hmm_features", dst="hmm_features")
      .update_variable("annsamps", ds.F(get_annsamples), mode='e')
      .update_variable("anntypes", ds.F(get_anntypes), mode='e')
      .update_variable("hmm_features", ds.B("hmm_features"), mode='e')
      .run(batch_size=20, shuffle=False, drop_last=False, n_epochs=1, lazy=True)
)

ppl_inits = (dtst >> template_ppl_inits).run()

In [6]:
lengths = [hmm_features.shape[2] for hmm_features in ppl_inits.get_variable("hmm_features")]
hmm_features = np.concatenate([hmm_features[0,:,:].T for hmm_features in ppl_inits.get_variable("hmm_features")])
anntype = ppl_inits.get_variable("anntypes")
annsamp = ppl_inits.get_variable("annsamps")

In [7]:
expanded = np.concatenate([expand_annotation(samp, types, length) for samp, types, length in zip(annsamp, anntype, lengths)])
means, covariances = prepare_means_covars(hmm_features, expanded, states = [3, 5, 11, 14, 17, 19], num_features = 3)
transition_matrix, start_probabilities = prepare_transmat_startprob()

In [11]:
#Training of model
config_train = {
    'build': True,
    'estimator': hmm.GaussianHMM(n_components=19, n_iter=25, covariance_type="full", random_state=42,
                                 init_params='', verbose=False),
    'init_params': {'means_': means, 'covars_': covariances, 'transmat_': transition_matrix,
                    'startprob_': start_probabilities}
                }

ppl_train_template = (
    ds.Pipeline()
      .init_model("dynamic", HMModel, "HMM", config=config_train)
      .load(fmt='wfdb', components=["signal", "annotation", "meta"], ann_ext='pu1')
      .cwt(src="signal", dst="hmm_features", scales=[4,8,16], wavelet="mexh")
      .standardize(axis=-1, src="hmm_features", dst="hmm_features")
      .train_model("HMM", make_data=partial(prepare_hmm_input, features="hmm_features", channel_ix=0))
      .run(batch_size=10, shuffle=False, drop_last=False, n_epochs=1, lazy=True)
)

# ppl_train = (
#   dtst.train
#       .pipeline  
#       .init_model("dynamic", HMModel, "HMM", config=config_train)
#       .load(fmt='wfdb', components=["signal", "annotation", "meta"], ann_ext='pu1')
#       .wavelet_transform_signal(cwt_scales=[4,8,16], cwt_wavelet="mexh")
#       .train_model("HMM", make_data=prepare_batch)
#       .run(batch_size=20, shuffle=False, drop_last=False, n_epochs=1)
# )

In [12]:
ppl_train = (dtst >> ppl_train_template).run()

In [13]:
ppl_train.save_model("HMM", path="./models/hmmodel.dill")